# $k$-means Clustering

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

## Initialization

In [2]:
dataset = load_iris()  # If you want to know more about this dataset: https://en.wikipedia.org/wiki/Iris_flower_data_set
data = dataset['data'][:, 2:]  # We use only the petal length and width
k = 3
iterations = 20

## Implement the Algorithm
### Functions

In [3]:
def reassign(prototypes):
    nbrs = NearestNeighbors(n_neighbors=1).fit(prototypes)
    indices = nbrs.kneighbors(data, return_distance=False)
    return indices[:,0]

def recalculate(indices):
    new_prototypes = []
    for p_ind,p in enumerate(cluster_prototypes[len(cluster_prototypes)-1]):
        sum =0;
        count = 0;
        for j,j_ind in enumerate(indices):
            if(j_ind == p_ind):
                sum += data[j]
                count += 1
        val = (1 / count) * sum
        new_prototypes.append(val)
    return np.array(new_prototypes)

def total_variance(indices, prototypes):
    new_variance = 0
    for p_ind,p in enumerate(prototypes):
        points_tmp = []
        for i,ind in enumerate(indices):
            if(ind == p_ind):
                points_tmp.append(data[i])
        new_variance += (min(distance.cdist(np.array(points_tmp),np.array([p]),'euclidean')))
    return new_variance      


### Run the Algorithm

In [4]:
np.random.seed(2)  # For reproducibility
cluster_prototypes = []
cluster_indices = []
variances = []

rand = np.random.randint(0, high=len(data)-1, size=k)
cluster_prototypes.append(np.array([data[rand[0]],data[rand[1]],data[rand[2]]]))
cluster_indices.append(reassign(cluster_prototypes[0]))
variances.append(total_variance(cluster_indices[0],cluster_prototypes[0]))

for t in range(1,iterations+1):
    cluster_prototypes.append(recalculate(cluster_indices[t-1]))
    cluster_indices.append(reassign(cluster_prototypes[t]))
    variances.append(total_variance(cluster_indices[t],cluster_prototypes[t]))


## Visualize the Result

In [5]:
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 6), gridspec_kw={'height_ratios': [2, 1]})
fig.subplots_adjust(hspace=0.35, top=0.95, bottom=0.1)

def plot_cluster(iteration):
    indices = cluster_indices[iteration]
    prototypes = cluster_prototypes[iteration]
    
    ax1.clear()
    ax2.clear()

    # Plot each cluster
    for i in range(k):
        points = data[indices == i, :]
        ax1.scatter(points[:, 0], points[:, 1], c='C{:d}'.format(i), s=15)
        ax1.scatter(prototypes[i, 0], prototypes[i, 1], c='C{:d}'.format(i), marker='D', s=50, edgecolor='k', linewidth=2)
    
    ax1.set_xlabel('Petal Lengths (cm)')
    ax1.set_ylabel('Petal Widths (cm)')
    ax1.set_title('Cluster result')
    ax1.set_axisbelow(True)
    ax1.grid()
    
    # Variance plot
    ax2.plot(variances)
    ax2.scatter(iteration, variances[iteration], c='gray', zorder=3)
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('$D_{var}$')
    ax2.set_title('Variance criterion')
    ax2.set_axisbelow(True)
    ax2.grid()

iteration_slider = widgets.IntSlider(min=0, max=iterations, description='Iteration:')
interact(plot_cluster, iteration=iteration_slider);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Iteration:', max=20), Output()), _dom_classes=('widget-i…